In [1]:
import pandas as pd

#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/10/16 04:31:11 WARN Utils: Your hostname, Rudyletsgo resolves to a loopback address: 127.0.1.1; using 172.31.44.254 instead (on interface eth0)
22/10/16 04:31:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/16 04:31:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/16 04:31:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/16 04:31:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/16 04:31:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/16 04:31:13 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
## read the merchant data
merchant_info = spark.read.options(header = True).csv('../data/curated/merchant_info.csv')
merchant_tents = merchant_info.where(merchant_info.field == "tent and awning shops")
merchant_jewl = merchant_info.where(merchant_info.field == "jewelry, watch, clock, and silverware shops")
merchant_shoe = merchant_info.where(merchant_info.field == "shoe shops")

# Transform to pandas dataframe
merchant_info = merchant_info.toPandas()
merchant_tents = merchant_tents.toPandas()
merchant_jewl = merchant_jewl.toPandas()
merchant_shoe = merchant_shoe.toPandas()

## convert dataset columns into fesible types
merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_tents[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_tents[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_jewl[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_jewl[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_shoe[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_shoe[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

#merchant_tents = merchant_info.loc[merchant_info['field'] == "tent and awning shops"].copy()
#merchant_jewl = merchant_info.loc[merchant_info['field']  == "jewelry, watch, clock, and silverware shops"].copy()
#merchant_shoe = merchant_info.loc[merchant_info['field'] == "shoe shops"].copy()

In [3]:
list(merchant_info.keys())

['merchant_abn',
 'transaction_count',
 'field',
 'take_rate',
 'revenue_level',
 'total_revenue',
 'mean_consumer_income',
 'fraud_count',
 'main_business_area_popu']

### Create score criteria(standard version)

In [4]:
## Manually set the score criteria, could be changed later.
score_criteria = { \
    'transaction_count': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
        'take_rate': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
            'revenue_level': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                'total_revenue': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

In [5]:
## store all numeric features into list for use in both Algorithms
featrue_name = list(merchant_info.keys())
featrue_name.remove('merchant_abn')
featrue_name.remove('field')

In [6]:
merchant_info

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,16903677169,51,shoe shops,4.62,b,18506.39,807.033332,0,103936
1,21359184622,18011,motor vehicle supplies and new parts,3.60,b,5659723.28,791.199817,3,93730
2,21782713770,294,tent and awning shops,3.31,b,336040.60,797.855442,0,108284
3,23692710924,3248,"florists supplies, nursery stock, and flowers",5.61,a,969325.83,792.499907,0,116502
4,25499263184,64,"books, periodicals, and newspapers",6.14,a,32355.13,807.621875,0,44114
...,...,...,...,...,...,...,...,...,...
4021,43833568675,318,telecom,4.69,b,814552.89,793.950000,1,274443
4022,56923368251,146,"watch, clock, and jewelry repair shops",6.14,a,25387.87,765.681506,0,107036
4023,59190126409,174,"florists supplies, nursery stock, and flowers",6.87,a,60788.54,778.840805,0,83204
4024,83177825742,9206,"gift, card, novelty, and souvenir shops",5.02,b,1012564.70,787.400760,5,37314


In [7]:
score_criteria

{'transaction_count': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'take_rate': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'revenue_level': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4},
 'total_revenue': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'mean_consumer_income': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'fraud_count': {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0},
 'main_business_area_popu': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

## Mark Algorithm

In [8]:
## Function used to assign continuous features into level (a,b,c,d,e) based on current merchant infomation
## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.

def mark_algorithm(merchant_info, feature, score_criteria):

    label_list = []
    score_list = []

    ## in each run of the ranking system, each numeric column is allocated with a level in (a,b,c,d,e)
    ## according to the overall quantile statistic of the feature column we are working on
    ab = merchant_info[feature].quantile(0.8)
    bc = merchant_info[feature].quantile(0.6)
    cd = merchant_info[feature].quantile(0.4)
    de = merchant_info[feature].quantile(0.2)
    
    for i in range(len(merchant_info)):
        if merchant_info.loc[i, feature] >= ab:
            label_list.append('a')
        elif (merchant_info.loc[i, feature] >= bc) & (merchant_info.loc[i, feature] < ab):
            label_list.append('b')
        elif (merchant_info.loc[i, feature] >= cd) & (merchant_info.loc[i, feature] < bc):
            label_list.append('c')
        elif (merchant_info.loc[i, feature] >= de) & (merchant_info.loc[i, feature] < cd):
            label_list.append('d')
        elif merchant_info.loc[i, feature] < de:
            label_list.append('e')

    ## mark each entry of this feature column according to the score_criteria given.
    for j in label_list:
        score_list.append(score_criteria[feature][j])

    return score_list

## Rank Algorithm

In [9]:
## Function used to rank mechant iteratively and then find the top 10 merchants to collaborate.

## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.
## remove_rate (a positive float {0,1}) is the rate to remove merchants classified as poor choice
## top_n (a positive integer) is the number of merchant to be recommended.

## Note: the lower the remove_rate, the higher the accuracy but also the longer the waiting time. (too low may cause overfitting)

def rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n):
    # Stop when we find the asked number of merchants as recommendation
    while (len(merchant_info) > top_n) & (len(merchant_info) * (1-remove_rate) > top_n):
        # In each run, set the score of all merchant to zero as a fresh analysis
        merchant_info['score'] = 0

        # The main part of our rank system to mark and sum scores of all feature columns 
        for feature in feature_name:
            if feature == 'revenue_level':
                for i in range(len(merchant_info)):
                    merchant_info.loc[i, 'score'] += score_criteria[feature][merchant_info.loc[i, feature]]
            else:
                # use of mark algorthm
                merchant_info['score'] += mark_algorithm(merchant_info, feature, score_criteria)

        # sort all merchants by their total score        
        merchant_info = merchant_info.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        # remove a percentage number of merchants having the lowest score (according to remove_rate given)
        merchant_info = merchant_info.drop(merchant_info.tail(int(len(merchant_info) * remove_rate)).index)
        # remove the score column after sorting and dropping merchants. (keep data integrity for next run of this algorithm)
        merchant_info = merchant_info.drop(['score'], axis=1)
        
        # if the stop criteria is not met, keep running to remove merchants that are less likely to be beneficial.
        return rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n)
    
    # Show the top n merchants once stop criteria are met.
    return merchant_info.head(top_n)

In [10]:
## Run Run Run!!!!!!!!!!
top_n_merchant = rank_algorithm(merchant_info, featrue_name, score_criteria, 0.1, 100)

In [11]:
top_n_merchant.head(10)

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,41944909975,29998,"books, periodicals, and newspapers",0.16,e,7333635.99,792.020831,6,163700
1,90568944804,9327,tent and awning shops,4.10,b,8377145.54,792.986845,87,200914
2,23540152063,1403,shoe shops,6.77,a,594838.53,799.302138,0,260227
3,93442141629,1025,"music shops - musical instruments, pianos, and...",6.53,a,509322.99,805.543025,0,261705
4,73761689784,859,motor vehicle supplies and new parts,4.17,b,1262563.04,807.837136,0,292974
5,45629217853,198936,"gift, card, novelty, and souvenir shops",6.98,a,7325106.13,791.062344,43,114659
6,98072939449,5181,artist supply and craft shops,6.89,a,3118273.24,792.552365,2,148050
7,49549583265,8756,tent and awning shops,5.69,a,5946645.18,789.880448,4,237414
8,77338620996,10803,"computers, computer peripheral equipment, and ...",3.59,b,5379965.48,790.344488,5,264014
9,99018524323,819,shoe shops,0.42,e,200208.73,800.855677,0,415959


In [12]:
# The summary of top 100 merchants
# Revenue level count
top_n_merchant.groupby('field').count()

,merchant_abn,transaction_count,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
field,,,,,,,,
"antique shops - sales, repairs, and restoration services",5,5,5,5,5,5,5,5
art dealers and galleries,1,1,1,1,1,1,1,1
artist supply and craft shops,8,8,8,8,8,8,8,8
bicycle shops - sales and service,4,4,4,4,4,4,4,4
"books, periodicals, and newspapers",3,3,3,3,3,3,3,3
"cable, satellite, and other pay television and radio services",3,3,3,3,3,3,3,3
"computer programming , data processing, and integrated systems design services",3,3,3,3,3,3,3,3
"computers, computer peripheral equipment, and software",4,4,4,4,4,4,4,4
"digital goods: books, movies, music",1,1,1,1,1,1,1,1


In [13]:
# The top 100 merchants take rate mean and median
print(top_n_merchant['take_rate'].mean())
print(top_n_merchant['take_rate'].median())
print("comparison to the original merchant")
print(merchant_info['take_rate'].mean())
print(merchant_info['take_rate'].median())

5.150000000000002
5.995
comparison to the original merchant
4.397575757575765
4.5


In [14]:
# The top 100 revenue mean and median
print(top_n_merchant['total_revenue'].mean())
print(top_n_merchant['total_revenue'].median())
print("comparison to the original merchant")
print(merchant_info['total_revenue'].mean())
print(merchant_info['total_revenue'].median())

2965031.3289000015
1239599.78
comparison to the original merchant
468184.5472305021
123231.16500000001


## Prepare the segment merchant data for ranking
### Three selected segments are "tent and awning shops", "telecom" and "shoe shops"

# Design a score criteria with high priority on take rate
## specifically for the segment jewlery

In [15]:
top_10_merchant_jewlery = rank_algorithm(merchant_jewl, featrue_name, score_criteria, 0.2, 10)

In [16]:
# This is the top 10 jewlery shops when using standard criteria
top_10_merchant_jewlery

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,15043504837,175,"jewelry, watch, clock, and silverware shops",4.62,b,2912189.09,803.456572,62,189833
1,19492220327,723,"jewelry, watch, clock, and silverware shops",4.93,b,7198756.77,784.865283,128,106115
2,90918180829,493,"jewelry, watch, clock, and silverware shops",2.62,c,5072774.57,786.765720,100,151359
3,12983555508,6,"jewelry, watch, clock, and silverware shops",5.09,b,55507.02,822.333333,1,299938
4,75265429612,16,"jewelry, watch, clock, and silverware shops",4.92,b,156113.21,832.812500,4,334730
5,41331227038,34,"jewelry, watch, clock, and silverware shops",3.97,b,157573.05,837.970586,0,158242
6,28639309109,183,"jewelry, watch, clock, and silverware shops",1.70,c,44958.93,797.785247,0,344095
7,57860746842,99,"jewelry, watch, clock, and silverware shops",4.64,b,1115039.24,819.185858,16,23844
8,86889657711,66,"jewelry, watch, clock, and silverware shops",4.65,b,1079196.73,816.012120,24,93494
9,93110588804,18,"jewelry, watch, clock, and silverware shops",3.86,b,224793.54,809.205556,3,224663


### Next want to see the rank when using designed score crietria

### Buiness rule
No.1: Merchant care more about the consumption ability of their target customers (High mean income & low population > low income & large population )

No.2: The BNPL company tends to select the merchants allow them to earn more (High take rate is prefered)

No.3: Combined with No.2, the BNPL company like the merchant has frequent transactions (Large transactions count is prefered)

No.3 alternative: BNPL company may like the merchant has large total revenue 

No.4: Large penalty on fraud probability as this segment is related to luxury goods

In [17]:
score_criteria_jewlery = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
            'revenue_level': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                'total_revenue': {'a': 6, 'b': 4.5, 'c': 3, 'd': 1.5, 'e': 0}, \
                    'mean_consumer_income': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3.5, 'c': 3, 'd': 2.5, 'e': 2}}

In [18]:
top_10_merchant_jewlery_designed = rank_algorithm(merchant_jewl, featrue_name, score_criteria_jewlery, 0.2, 10)

In [19]:
top_10_merchant_jewlery_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,19492220327,723,"jewelry, watch, clock, and silverware shops",4.93,b,7198756.77,784.865283,128,106115
1,15043504837,175,"jewelry, watch, clock, and silverware shops",4.62,b,2912189.09,803.456572,62,189833
2,44345785419,81,"jewelry, watch, clock, and silverware shops",6.07,a,1211300.90,796.754320,23,82628
3,57860746842,99,"jewelry, watch, clock, and silverware shops",4.64,b,1115039.24,819.185858,16,23844
4,90918180829,493,"jewelry, watch, clock, and silverware shops",2.62,c,5072774.57,786.765720,100,151359
5,75265429612,16,"jewelry, watch, clock, and silverware shops",4.92,b,156113.21,832.812500,4,334730
6,21073645356,20,"jewelry, watch, clock, and silverware shops",5.75,a,138378.86,825.215002,1,86255
7,41331227038,34,"jewelry, watch, clock, and silverware shops",3.97,b,157573.05,837.970586,0,158242
8,28639309109,183,"jewelry, watch, clock, and silverware shops",1.70,c,44958.93,797.785247,0,344095
9,10596295795,97,"jewelry, watch, clock, and silverware shops",6.84,a,1122230.91,769.343299,19,86562


## Specifically for the segment shoe shops

### First see the rank using standard score critria

In [20]:
top_10_merchant_shoe = rank_algorithm(merchant_shoe, featrue_name, score_criteria, 0.2, 10)

In [21]:
top_10_merchant_shoe

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,99018524323,819,shoe shops,0.42,e,200208.73,800.855677,0,415959
1,23540152063,1403,shoe shops,6.77,a,594838.53,799.302138,0,260227
2,56796971172,3168,shoe shops,5.81,a,1291325.20,795.960512,1,142522
3,66678764219,3724,shoe shops,4.47,b,1222621.59,791.248791,0,274788
4,98548263806,7728,shoe shops,2.91,c,1248654.67,796.478235,0,71617
5,66842618444,4616,shoe shops,6.93,a,1133102.58,794.988019,3,91766
6,47459279421,4374,shoe shops,6.86,a,1063231.66,793.816483,2,127244
7,76351957431,70,shoe shops,6.20,a,23745.36,811.985713,0,289705
8,85362139954,3014,shoe shops,6.14,a,1270944.01,789.010915,1,162071
9,15912561882,7374,shoe shops,3.72,b,1221923.44,794.264633,2,114917


### The top 10 shoe shops when applying designed score criteria

### Business rule
No.1: Merchant care more about the numbers of potential customers (low mean income & large population > High income & small population )

No.2: The BNPL company tends to select the merchants considered to be stable (large total transactions count is prefered)

No.3 small penalty on fraud probability as shoes are usually considered to be affordable

In [22]:
score_criteria_shoe = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
            'revenue_level': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                'total_revenue': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3.5, 'c': 3, 'd': 2.5, 'e': 2}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}}

In [23]:
top_10_merchant_shoe_designed = rank_algorithm(merchant_shoe, featrue_name, score_criteria_shoe, 0.2, 10)

In [24]:
top_10_merchant_shoe_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,11439466003,26038,shoe shops,5.71,a,4246363.75,791.043774,1,100128
1,88553358884,8533,shoe shops,6.38,a,855698.67,791.866261,0,128974
2,66678764219,3724,shoe shops,4.47,b,1222621.59,791.248791,0,274788
3,15912561882,7374,shoe shops,3.72,b,1221923.44,794.264633,2,114917
4,99018524323,819,shoe shops,0.42,e,200208.73,800.855677,0,415959
5,85362139954,3014,shoe shops,6.14,a,1270944.01,789.010915,1,162071
6,56796971172,3168,shoe shops,5.81,a,1291325.20,795.960512,1,142522
7,93558142492,19080,shoe shops,3.41,b,7737603.79,790.789450,108,65863
8,23540152063,1403,shoe shops,6.77,a,594838.53,799.302138,0,260227
9,76351957431,70,shoe shops,6.20,a,23745.36,811.985713,0,289705


## Specifically for the segment tent shops

Standard score criteria

In [25]:
top_10_merchant_tents = rank_algorithm(merchant_tents, featrue_name, score_criteria, 0.2, 10)

In [26]:
top_10_merchant_tents

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,90568944804,9327,tent and awning shops,4.10,b,8377145.54,792.986845,87,200914
1,38700038932,6244,tent and awning shops,6.31,a,8350160.13,793.383616,0,62683
2,89726005175,188223,tent and awning shops,6.01,a,7764006.97,790.796524,37,116241
3,46331355995,1728,tent and awning shops,6.27,a,769239.96,799.922107,1,164115
4,62787106107,650,tent and awning shops,2.87,c,922483.62,794.436001,0,187875
5,88299699256,42,tent and awning shops,6.75,a,19696.54,810.611904,0,173099
6,49549583265,8756,tent and awning shops,5.69,a,5946645.18,789.880448,4,237414
7,80551528183,6813,tent and awning shops,4.81,b,7673689.76,789.393292,3,187544
8,49891706470,215786,tent and awning shops,5.80,a,6254617.02,791.043793,49,144951
9,89640578182,16013,tent and awning shops,6.22,a,464899.04,792.425761,4,145198


### Designed score criteria


### Business rule

No.1: The BNPL company tends to select the merchants considered to be stable (large total transactions count and large total revenue is prefered)

No.2 large penalty on fraud probability as clients dont like the risks

In [27]:
# Higher priority means the algorithm tends to select the merchant that can make the BNPL company earn more on each transaction
score_criteria_tents = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':40, 'b': 30, 'c': 20, 'd': 10, 'e': 0}, \
            'revenue_level': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                'total_revenue': {'a': 8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

In [28]:
top_10_merchant_tents_designed = rank_algorithm(merchant_tents, featrue_name, score_criteria_tents, 0.2, 10)

In [29]:
top_10_merchant_tents_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,70610974780,13995,tent and awning shops,6.93,a,577739.46,791.886767,2,61750
1,22644977942,195,tent and awning shops,6.96,a,175906.12,789.651795,0,104446
2,96911331900,383,tent and awning shops,6.88,a,433536.95,777.498694,0,162632
3,34703763702,22,tent and awning shops,6.98,a,23989.62,743.218181,0,52582
4,66432686430,1650,tent and awning shops,6.81,a,1123207.41,781.653757,0,50817
5,27573216953,40,tent and awning shops,6.82,a,15127.18,789.457503,0,82736
6,63523606379,305,tent and awning shops,6.80,a,199957.24,782.634753,0,108799
7,51221851716,419,tent and awning shops,6.77,a,11955.38,802.758950,0,117191
8,64203420245,227504,tent and awning shops,2.86,c,6584186.21,791.394502,37,114659
9,92007575783,112,tent and awning shops,6.73,a,141954.03,786.773215,0,106548
